# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [56]:
import scipy.stats as stats
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [3]:
df = pd.read_csv("homepage_actions.csv")
df

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view
...,...,...,...,...
8183,2017-01-18 09:11:41.984113,192060,experiment,view
8184,2017-01-18 09:42:12.844575,755912,experiment,view
8185,2017-01-18 10:01:09.026482,458115,experiment,view
8186,2017-01-18 10:08:51.588469,505451,control,view


In [17]:
df['action'].unique()

array(['view', 'click'], dtype=object)

In [18]:
df['action'].value_counts()

view     6328
click    1860
Name: action, dtype: int64

In [42]:
v_df = df.loc[df['action'] == 'view', ['id', 'action']]
c_df = df.loc[df['action'] == 'click', ['id', 'action']]
v = len(v_df['id'].unique())
c = len(c_df['id'].unique())
print(v)
print(c)
"""
There are less clickers than viewers therefore everyone who viewed didn't click. 
"""
print(v - c)
#4468 non clicking viewers
print(c - v)
#-4468 in not possible so all clicker viewed
#so the amount of viewers who clicked are the total number of clickers

6328
1860
4468
-4468


In [51]:
e_df = df.loc[df['group'] == 'experiment', 'id']
c_df = df.loc[df['group'] == 'control', 'id']
e = e_df.unique()
c = c_df.unique()
e_c = []
for x in e:
    for y in c:
        if x == y:
            e_c.append(1)

if sum(e_c) > 0:
    print('Overlap')
else:
    print('No Overlap')
            

No Overlap


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [52]:
"""
Because we are comparing the what the two groups did in terms of actions, that seems to be categorical
in nature to me give the data.  Therefore a Chi-squared test would be best. Making a matrix of 
the two groups by the actions they took and then running the test.
"""

'\nBecause we are comparing the what the two groups did in terms of actions, that seems to be categorical\nin nature to me give the data.  Therefore a Chi-squared test would be best. Making a matrix of \nthe two groups by the actions they took and then running the test.\n'

In [58]:
"""
H0:There is no significant difference between the experimental group and the control group
HA:There is a difference between the two groups
Alpha: using the standard .05 alpha
"""

'\nH0:There is no significant difference between the experimental group and the control group\nHA:There is a difference between the two groups\nAlpha: using the standard .05 alpha\n'

In [54]:
chi_table = pd.crosstab(df['group'], df['action'])
chi_table

action,click,view
group,,
control,932,3332
experiment,928,2996


In [57]:
chi2, p, dof, ex = stats.chi2_contingency(chi_table)

print("Chi-square statistic:", chi2)
print("p-value:", p)

Chi-square statistic: 3.636160051233291
p-value: 0.056537191086915774


In [59]:
#Given this result we would fail to reject the H0

## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [75]:
cg_c = df.loc[df['group'] == 'control', ['id', 'action']]
cg_c = cg_c[cg_c.action == 'click']
c_group = df.loc[df['group'] == 'control', 'id']
ctr = cg_c.groupby(['id']).count().sum() / len(c_group)
ctr

e_group = df.loc[df['group'] == 'experiment', 'id']
ex_clk_num = ctr * len(e_group)
ex_clk_num


action    857.684803
dtype: float64

### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [79]:
n = len(e_group)
p = ctr
var = n * p * (1-p)
std = np.sqrt(var)
std

ex_clk = df.loc[df['group'] == 'experiment', ['id', 'action']]
ex_clk = ex_clk[ex_clk.action == 'click']
ac_ex_clk_num = ex_clk.groupby(['id']).count().sum()
ac_ex_clk_num

z_score = (ac_ex_clk_num - ex_clk_num)/std
z_score

action    2.716073
dtype: float64

### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [81]:
p_val = 1 - stats.norm.cdf(z_score)
p_val

array([0.00330307])

### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: No, probably because I chose to do a chi-squared test.

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.